In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator  # Updated import for AerSimulator
from qiskit.quantum_info import SparsePauliOp  # Removed unused Pauli import
 # Fixed module path
import numpy as np
from qiskit.primitives import (
    BaseSamplerV1,
    StatevectorSampler,
    BaseEstimatorV1,
    StatevectorEstimator,
)
 


In [2]:

def create_qubit_hamiltonian(num_qubits: int) -> SparsePauliOp:
    pauli_list = []
    for i in range(num_qubits - 1):
        label = "I" * i + "ZZ" + "I" * (num_qubits - i - 2)
        pauli_list.append((label, 0.5))
    for i in range(num_qubits):
        label = "I" * i + "X" + "I" * (num_qubits - i - 1)
        pauli_list.append((label, 0.2))
    for i in range(num_qubits):
        label = "I" * i + "Z" + "I" * (num_qubits - i - 1)
        pauli_list.append((label, 0.1))
    return SparsePauliOp.from_list(pauli_list)
num_qubits = 10
hamiltonian = create_qubit_hamiltonian(num_qubits)

In [3]:

# Step 2: Define the adaptable ansatz with num_qubits parameter
def create_ansatz(num_qubits, params, depth=1, connectivity="linear"):
    """
    Creates an ansatz circuit for a variable number of qubits.
    Args:
        num_qubits: Number of qubits in the circuit
        params: Array of rotation angles (length = num_qubits * depth)
        depth: Number of layers (default 1)
        connectivity: "linear" (nearest-neighbor) or "cyclic" (includes last->first)
    Returns:
        QuantumCircuit object
    """
    qc = QuantumCircuit(num_qubits)
    
    # Ensure params matches the expected size
    expected_params = num_qubits * depth
    if len(params) != expected_params:
        raise ValueError(f"Expected {expected_params} parameters, got {len(params)}")
    
    # Build the circuit layer by layer
    for d in range(depth):
        # RY gates for each qubit in this layer
        for i in range(num_qubits):
            qc.ry(params[d * num_qubits + i], i)
        
        # Entangling CNOTs
        if connectivity == "linear":
            for i in range(num_qubits - 1):
                qc.cx(i, i + 1)
        elif connectivity == "cyclic":
            for i in range(num_qubits - 1):
                qc.cx(i, i + 1)
            qc.cx(num_qubits - 1, 0)  # Close the loop
        else:
            raise ValueError("Connectivity must be 'linear' or 'cyclic'")
    
    return qc

# Step 3: Compute the expectation value E(theta) using StatevectorEstimator
def compute_expectation(params):# Match the Hamiltonian
    estimator = StatevectorEstimator()
    circuit = create_ansatz(num_qubits, params, depth=1, connectivity="linear")
    job = estimator.run([(circuit, hamiltonian)])
    return job.result()[0].data.evs

# Step 4: SPSA optimizer (unchanged)
def spsa_optimizer(objective_function, initial_params, maxiter=100, a=0.01, c=0.1, alpha=0.602, gamma=0.101):
    params = np.copy(initial_params)
    n = len(params)
    for k in range(maxiter):
        ak = a / (k + 1) ** alpha
        ck = c / (k + 1) ** gamma
        delta = np.random.choice([-1, 1], size=n)
        params_plus = params + ck * delta
        params_minus = params - ck * delta
        y_plus = objective_function(params_plus)
        y_minus = objective_function(params_minus)
        gradient = (y_plus - y_minus) / (2 * ck * delta)
        params -= ak * gradient
    optimal_value = objective_function(params)
    return params, optimal_value

# Step 5: Run VQE
  # Match the Hamiltonian
depth = 1
initial_params = np.random.uniform(0, 2 * np.pi, num_qubits * depth)  # 30 params for depth=1
optimal_params, optimal_energy = spsa_optimizer(
    objective_function=compute_expectation,
    initial_params=initial_params,
    maxiter=100
)

# Step 6: Output results
print(f"Optimal parameters: {optimal_params}")
print(f"Optimal energy: {optimal_energy}")
# Optional: Exact ground state for comparison (using numpy)
H_matrix = hamiltonian.to_matrix()
eigenvalues = np.linalg.eigvalsh(H_matrix)
exact_energy = eigenvalues[0]
print(f"Exact ground state energy: {exact_energy}")

Optimal parameters: [4.08389463 4.75519643 5.04737115 5.84990654 1.38980717 4.92104413
 1.14018609 4.75076171 4.63080539 1.45987271]
Optimal energy: 1.0076012105295067
Exact ground state energy: -4.7475101024045125
